# TIR Collection

## Loading relevant libraries

In [ ]:
library(tidyverse)
library(rGV)

## Dexcom
Given that the exported CSV files for Dexcom and Freestyle Libre are formatted differently, reading and wrangling must also be carried out differently. Following this, the process for TIR collection is the same for both CGMs.

### Defining Function

In [ ]:
clean_file <- function(file) {
    suppressWarnings ({
        read <- read_csv(file, show_col_types = FALSE) 
        
        # renaming columns for ease of modifying rows and columns
        names(read)[8] = 'Glucose_Value'
        names(read)[2] = 'Timestamp'
        names(read)[14] = 'Transmitter_ID'
        
        # removing top 11 rows that contain user name and alert types (irrelevant) and removing glucose values that are NA
        rows <- filter(read, Transmitter_ID != 'NA') %>%
                filter(Glucose_Value != 'NA')
        # selecting only the timestamp and glucose value columns that will be relevant to calculate statistics
        cols <- select(rows, Timestamp, Glucose_Value)
        
        # pulling vectors from columns to perform calculations
        glucose_vector_str <- cols %>% pull(Glucose_Value)
        glucose_vector <- as.double(glucose_vector_str) # glucose values were in strings, converting to decimal values for calculations
        time_vector <- cols %>% pull(Timestamp)
        times <- minute(time_vector) # pulling only minutes from timestamps for compatibility with calculations
        
        # using the rGV library to perform the following calculations                
        mag <- mag(x=glucose_vector, times=times) # mean absolute glucose
        cv <- cv(x=glucose_vector, times=times, overall=TRUE) # coefficient of variant
        sd <- st_dev(x=glucose_vector, times=times, overall=TRUE) # standard deviation
        tir <- tir(x=glucose_vector, low=4.0, high=10.0) # time in range
        tbr <- tir(x=glucose_vector, low=0.0, high=3.9) # time below range
        tar <- tir(x=glucose_vector, low=10.1, high=25.0) # time above range

        # creating a data frame with the statistics calculated
        df <- data.frame(tir, tbr, tar, mag, cv, sd)
    })
}

### Applying Function
Calling the `clean_file` function on all .csv files in the Dexcom folder, and writing the data into a final .csv file.

In [ ]:
output_file <- 'tir_collection/test_df.csv'

final_data <- list.files(path = 'tir_collection/data/Dexcom',    
                       pattern = "*.csv", full.names = TRUE) %>% 
  lapply(clean_file) %>%                                           
  bind_rows                                                      
  
final_data
write.csv(final_data, output_file)